##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: OAuth 快速入門


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/zh-tw/quickstarts/Authentication_with_OAuth.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


Gemini API 的某些部分，例如模型微調和語義檢索，使用 OAuth 進行驗證。

如果你是初學者，你應該從使用 [API 金鑰](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 開始，並僅在需要這些功能時再回到此 OAuth 指南。

為了幫助你開始使用 OAuth，此筆記本展示了一種簡化的方式，適用於測試環境。

對於生產環境，在 [選擇適合你應用程式之存取資格證](https://developers.google.com/workspace/guides/create-credentials#choose_the_access_credential_that_is_right_for_you) 前，了解有關 [驗證和授信](https://developers.google.com/workspace/guides/auth-overview) 的資訊。


## 執行條件

若要運行這個快速入門，你需要：

*   安裝在本地端的 [Google Cloud CLI](https://cloud.google.com/sdk/docs/install-sdk) 。
*   [Google Cloud 專案](https://developers.google.com/workspace/guides/create-project) 。

如果你在 Google AI Studio 中建立了 API 金鑰，則會為你建立 Google Cloud 專案。前往 [Google AI Studio](https://aistudio.google.com/app/apikey) 並記錄下 Google Cloud 專案名稱以使用該專案。


## 設定你的 Cloud 專案

若要完成這份快速入門指南，你需要先設定 Cloud 專案。

### 1. 啟用 API

在使用 Google API 之前，你需要在 Google Cloud 專案中啟用它們。

* 在 Google Cloud 主控台，[啟用](https://console.cloud.google.com/flows/enableapi?apiid=generativelanguage.googleapis.com) Google Generative Language API。如果你已在 AI 工作坊中建立 API 金鑰，則已為你完成這項工作。

### 2. 設定 OAuth 同意畫面

接著設定專案的 OAuth 同意畫面，並將你自己新增為測試使用者。如果你已為 Cloud 專案完成這項步驟，請跳到下一個章節。

1. 在 Google Cloud 主控台，前往 [OAuth 同意畫面](https://console.cloud.google.com/apis/credentials/consent)，可在 **功能表** > **API 和服務** > **OAuth 同意畫面** 中找到它。

2. 為應用程式選取 **外部** 使用者類型，然後按一下 **建立** 。

3. 填寫應用程式註冊表 (你可以將大部分欄位留空)，然後按一下 **儲存並繼續** 。

4. 目前你可以跳過新增範圍並按一下 **儲存並繼續** 。日後，當你為在 Google Workspace 組織外部使用的應用程式建立應用程式時，必須新增並驗證應用程式所需的授權範圍。

5. 新增測試使用者：
    1. 在 **測試使用者** 下，按一下 **新增使用者** 。
    2. 輸入你的電子郵件地址和任何其他已授權測試使用者，然後按一下 **儲存並繼續** 。

6. 檢閱你的應用程式註冊摘要。要進行變更，請按一下 **編輯** 。如果應用程式註冊看起來沒問題，請按一下 **返回儀表板** 。

### 3. 授權桌面應用程式的憑證

若要進行驗證並在應用程式中存取使用者資料，你需要建立一個或多個 OAuth 2.0 用戶端 ID。用戶端 ID 用於向 Google 的 OAuth 伺服器識別單一應用程式。如果應用程式在多個平台上執行，你必須為每個平台建立個別的用戶端 ID。

1. 在 Google Cloud 主控台，前往 [憑證](https://console.cloud.google.com/apis/credentials/consent)，可在 **功能表** > **API 和服務** > **憑證** 中找到它。

2. 按一下 **建立憑證** > **OAuth 用戶端 ID** 。
3. 按一下 **應用程式類型** > **桌面應用程式** 。
4. 在 **名稱** 欄位中，為憑證輸入名稱。這個名稱只會顯示在 Google Cloud 主控台。
5. 按一下 **建立** 。將出現已建立的 OAuth 用戶端畫面，顯示你的新用戶端 ID 和用戶端密碼。
6. 按一下 **確定** 。新建立的憑證會顯示在 **OAuth 2.0 用戶端 ID** 下。
7. 按一下下載按鈕儲存 JSON 檔案。它會儲存為 `client_secret_<識別碼>.json`。


## 設定應用程式預設憑證

在此快速入門中，你將使用 [應用程式預設憑證](https://cloud.google.com/docs/authentication/application-default-credentials) 進行驗證。


### 將客戶端金鑰加入 Colab 金鑰

如果你需要在 Google Colab 中頻繁使用 OAuth 和 Gemini API，建議你將 `client_secret.json` 檔案的內容新增到 Colab 的 Secrets 管理員。

1. 開啟你的 Google Colab 筆記本，並在左側面板中按一下 🔑 **Secrets** 標籤。
2. 輸入名稱`CLIENT_SECRET`，建立新的金鑰。
3. 在文字編輯器中開啟 `client_secret.json` 檔案，並將內容複製/貼上到 `CLIENT_SECRET` 的`Value` 輸入欄位中。
4. 切換左側的按鈕，以允許筆記本取得金鑰存取權。

現在，你可以程式化建立檔案，而不是每次都上傳。當你允許存取後，客戶端金鑰也會在所有 Google Colab 筆記本中提供。


In [ ]:
from google.colab import userdata
import pathlib
pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

413

### 設定應用程式預設憑證

要將 `client_secret.json` 檔案轉換成可用的憑證，請將其位置傳遞給 `gcloud auth application-default login` 指令的 `--client-id-file` 引數。

本教學課程中簡化的專案設定會觸發 **Google 尚未驗證這個應用程式** 對話方塊。這是正常的，請選擇 **繼續** 。

對於每一個新的 Google Colab 筆記本或執行階段，你都需要執行這個步驟一次。

**注意：** 仔細遵循下列指令列印的指示 (不要只按一下連結)。還要確保你本地的 `gcloud --version` 是 [最新版本](https://cloud.google.com/sdk/docs/release-notes)，以符合 Google Colab 預先安裝的版本。


In [ ]:
!gcloud auth application-default login \
  --no-browser --client-id-file client_secret.json \
  --scopes https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning,https://www.googleapis.com/auth/generative-language.retriever


你需要的特定 `scopes` 取決於你使用的 API。例如，查看 [`tunedModels.create`](https://ai.google.dev/api/rest/v1beta/tunedModels/create#authorization-scopes) 的 API 參考，你將看到：

> 需要以下其中一個 OAuth 範圍：
>
> * `https://www.googleapis.com/auth/generative-language.tuning`

此範例要求所有調整和語意檢索範圍，但最佳做法是使用最小的範圍集以確保安全性並提升使用者信心。


## 使用帶有 OAuth 的 Python SDK

Python SDK 將會自動查找並使用應用程式的預設憑據。


In [ ]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.7 MB/s eta 0:00:00


讓我們做一個快速測試。請注意，你沒有使用 `genai.configure()` 設定 API 金鑰！


In [ ]:
import google.generativeai as genai

print('Available base models:', [m.name for m in genai.list_models()])

# 附錄


## 從 Colab 進行經過驗證的 REST 呼叫

通常，建議在可行時使用 Python SDK 與 Gemini API 互動。此範例說明如何從 Python 進行 OAuth 經過驗證的 REST 呼叫，以進行偵錯或測試。假設你已從快速入門設定應用程式預設認證資訊。


In [ ]:
import requests

access_token = !gcloud auth application-default print-access-token

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {access_token[0]}',
}

response = requests.get('https://generativelanguage.googleapis.com/v1/models', headers=headers)
response_json = response.json()

# All the model names
for model in response_json['models']:
    print(model['name'])

### 分享調整過的模型

某些 Beta 版 API 功能可能尚不受 Python SDK 支援。此範例說明如何從 Python 執行 REST 呼叫，以加入調整過的模型權限。


In [ ]:
import requests

model_name = ''   # @param {type:"string"}
emailAddress = '' # @param {type:"string"}


access_token = !gcloud auth application-default print-access-token

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {access_token[0]}',
}

body = {
  'granteeType': 'USER',        # Or 'GROUP' or 'EVERYONE' https://ai.google.dev/api/rest/v1beta/tunedModels.permissions
  'emailAddress': emailAddress, # Optional if 'granteeType': 'EVERYONE'
  'role': 'READER'
}

response = requests.post(f'https://generativelanguage.googleapis.com/v1beta/tunedModels/{model_name}/permissions', json=body, headers=headers)
print(response.json())


## 使用服務帳戶進行身份驗證

Google Cloud [服務帳戶](https://cloud.google.com/iam/docs/service-account-overview) 是不代表人類用戶的帳戶。當人類沒有直接參與其中時，它們提供了一種管理驗證和授權的方式，例如你的應用程式呼叫 Gemini API 來完成使用者要求，但未驗證身分為使用者。使用服務帳戶對 Gemini API 進行驗證的一個簡單方法是使用 [服務帳戶金鑰](https://cloud.google.com/docs/authentication/provide-credentials-adc#local-key)。

本指南簡要說明如何在 Google Colab 中使用服務帳戶金鑰。

**重要事項：** 服務帳戶金鑰可能是安全風險！如需更多資訊，請參閱 [管理服務帳戶金鑰的最佳實務](https://cloud.google.com/iam/docs/best-practices-for-managing-service-account-keys)。

### 1. 建立服務帳戶

按照說明建立 [服務帳戶](https://cloud.google.com/iam/docs/service-accounts-create#creating)。如果你手動執行此操作，**主控台** 說明是最簡單的。

### 2. 建立服務帳戶金鑰

按照說明建立 [服務帳戶金鑰](https://cloud.google.com/iam/docs/keys-create-delete#creating)。請注意已下載金鑰的名稱。

### 3. 將服務帳戶金鑰新增到 Colab

 1. 開啟你的 Google Colab 筆記本，然後按一下左側面板中的 🔑 **機密** 標籤。
 2. 建立一個名稱為 `SERVICE_ACCOUNT_KEY` 的新機密。
 3. 在文字編輯器中開啟服務帳戶金鑰檔案，然後將內容複製/貼到 `SERVICE_ACCOUNT_KEY` 的 `值` 輸入方塊中。 
 4. 打開左側的按鈕，讓筆記本可以存取機密。

### 4. 使用服務帳戶金鑰對 Python SDK 進行身份驗證


In [ ]:
import google.generativeai as genai
import pathlib
from google.colab import userdata
from google.oauth2 import service_account

pathlib.Path('service_account_key.json').write_text(userdata.get('SERVICE_ACCOUNT_KEY'))

credentials = service_account.Credentials.from_service_account_file('service_account_key.json')

# Adjust scopes as needed
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/generative-language.retriever'])

genai.configure(credentials=scoped_credentials)

print('Available base models:', [m.name for m in genai.list_models()])